Most often, you would use optimization to search through the lever space in order to find promising policies. However, we can also use optimization to search through the uncertainty space in order to find for example a worst case scenario. 

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import copy

In [3]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time



ema_logging.log_to_stderr(ema_logging.INFO)

<Logger EMA (DEBUG)>

## Model Declaration

In [18]:
dike_model, planning_steps = get_model_for_problem_formulation(1)
uncertainties = copy.deepcopy(dike_model.uncertainties)
levers = copy.deepcopy(dike_model.levers)
outcomes= [i for i in dike_model.outcomes]

In [19]:
outcomes

[ScalarOutcome('Expected Annual Damage', variable_name=('A.1_Expected Annual Damage', 'A.2_Expected Annual Damage', 'A.3_Expected Annual Damage', 'A.4_Expected Annual Damage', 'A.5_Expected Annual Damage'), function=<function sum_over at 0x000002021E8F8400>),
 ScalarOutcome('Total Investment Costs', variable_name=('A.1_Dike Investment Costs', 'A.2_Dike Investment Costs', 'A.3_Dike Investment Costs', 'A.4_Dike Investment Costs', 'A.5_Dike Investment Costs', 'RfR Total Costs', 'Expected Evacuation Costs'), function=<function sum_over at 0x000002021E8F8400>),
 ScalarOutcome('Expected Number of Deaths', variable_name=('A.1_Expected Number of Deaths', 'A.2_Expected Number of Deaths', 'A.3_Expected Number of Deaths', 'A.4_Expected Number of Deaths', 'A.5_Expected Number of Deaths'), function=<function sum_over at 0x000002021E8F8400>)]

In [6]:
#with MultiprocessingEvaluator(dike_model) as evaluator:
with SequentialEvaluator(dike_model) as evaluator:
    results = evaluator.optimize(nfe=5000, epsilons=[0.25]*len(outcomes))

  0%|                                                 | 0/5000 [00:00<?, ?it/s][MainProcess/ERROR] cannot access local variable 'Qpeaks' where it is not associated with a value
Traceback (most recent call last):
  File "C:\Users\gabby\miniconda3\envs\MBDM\Lib\site-packages\ema_workbench\em_framework\experiment_runner.py", line 92, in run_experiment
    model.run_model(scenario, policy)
  File "C:\Users\gabby\miniconda3\envs\MBDM\Lib\site-packages\ema_workbench\util\ema_logging.py", line 153, in wrapper
    res = func(*args, **kwargs)
          ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\gabby\miniconda3\envs\MBDM\Lib\site-packages\ema_workbench\em_framework\model.py", line 347, in run_model
    outputs = self.run_experiment(experiment)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\gabby\miniconda3\envs\MBDM\Lib\site-packages\ema_workbench\util\ema_logging.py", line 153, in wrapper
    res = func(*args, **kwargs)
          ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\gabby\minicon

EMAError: Exception in run_model
Caused by: UnboundLocalError: cannot access local variable 'Qpeaks' where it is not associated with a value

#### Parallel Coordinate Plot

In [ ]:
from ema_workbench.analysis import parcoords

limits = parcoords.get_limits(outcomes)
axes = parcoords.ParallelAxes(limits)
axes.plot(outcomes)

# we invert this axis so direction of desirability is the same 
axes.invert_axis('max_P') 
plt.show()


In [ ]:
outcomes = results.loc[:, ['max_P', 'utility', 'reliability', 'inertia']]

limits = parcoords.get_limits(outcomes)
axes = parcoords.ParallelAxes(limits)
axes.plot(outcomes)

# we invert this axis so direction of desirability is the same 
axes.invert_axis('max_P') 
plt.show()

#### Convergence Assessment

In [ ]:
from ema_workbench.em_framework.optimization import (ArchiveLogger,
                                                     EpsilonProgress)
from ema_workbench import Constraint

#specify outcomes 
lake_model.outcomes = [ScalarOutcome('max_P', kind=ScalarOutcome.MINIMIZE,
                                     expected_range=(0,5)),
                       ScalarOutcome('utility', kind=ScalarOutcome.MAXIMIZE,
                                     expected_range=(0,2)),
                       ScalarOutcome('inertia', kind=ScalarOutcome.MAXIMIZE,
                                    expected_range=(0,1)),
                       ScalarOutcome('reliability', kind=ScalarOutcome.MAXIMIZE,
                                     expected_range=(0,1))]

convergence_metrics = [ArchiveLogger(
                        "./archives",
                        [l.name for l in lake_model.levers],
                        [o.name for o in lake_model.outcomes],
                        base_filename="tutorial.tar.gz",
                        ),
                        EpsilonProgress(),
                        ]


constraints = [Constraint("max pollution", outcome_names="max_P",
                          function=lambda x:max(0, x-5))]

In [ ]:
with MultiprocessingEvaluator(lake_model) as evaluator:
    results, convergence = evaluator.optimize(nfe=5000, searchover='levers',
                                    epsilons=[0.125, 0.05, 0.01, 0.01],
                                    convergence=convergence_metrics,
                                    constraints=constraints)


## Seed Analysis

In [ ]:
results = []
convergences = []
        
with MultiprocessingEvaluator(lake_model) as evaluator:
    # we run again for 5 seeds
    for i in range(5):
        # we create 2 covergence tracker metrics
        # the archive logger writes the archive to disk for every x nfe
        # the epsilon progress tracks during runtime
        convergence_metrics = [
            ArchiveLogger(
                "./archives",
                [l.name for l in lake_model.levers],
                [o.name for o in lake_model.outcomes],
                base_filename=f"{i}.tar.gz",
            ),
            EpsilonProgress(),
        ]

        # 5000 runs is clearly way to low, given the convergence 
        # analysis above. this is only for demonstration purposes
        result, convergence = evaluator.optimize(nfe=5000, searchover='levers',
                                     epsilons=[0.125, 0.05, 0.01, 0.01],
                                     constraints=constraints,
                                     convergence=convergence_metrics,)        
 
        results.append(result)
        convergences.append(convergence)        